# RQ1: Benchmark runtime

In [8]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import (
    bootstrap,
    gmean,
)
from IPython.display import display

from duet.process import *
from duet.constants import *
from common import *

df_raw = load_raw()
unique_suites = df_raw[RF.suite].unique()
df_raw.describe()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,total_ms,iteration_time_ns,process_cpu_time_ns,compilation_time_ms,compilation_total_ms,jmx_memory_young_collection_count,jmx_memory_young_collection_delta,jmx_memory_young_collection_total_ms,jmx_memory_young_collection_time_ms,jmx_memory_old_collection_count,...,jmx_memory_used_size,jmx_memory_used_delta,runid,iteration,iteration_start_ns,iteration_end_ns,epoch_start_ms,uptime_ns,start_unix,iteration_time
count,697583.000000,1.038841e+06,6.975830e+05,697583.000000,697583.000000,697583.000000,697583.000000,697583.000000,697583.000000,697583.000000,...,6.975830e+05,6.975830e+05,1.038841e+06,1.038841e+06,1.038841e+06,1.038841e+06,3.222370e+05,3.222370e+05,2.608690e+05,1.038841e+06
mean,90785.441795,9.200866e+09,3.526406e+09,514.675948,27636.055315,236.345903,5.307287,2618.687376,72.626959,41.067400,...,1.556221e+08,1.290187e+08,9.260301e-01,3.872330e+01,1.657931e+18,1.657931e+18,1.657906e+12,1.662959e+11,1.661841e+12,8.720431e+00
std,95573.484559,4.507282e+10,3.859958e+09,2380.145943,30282.763530,394.289155,10.865766,7097.813296,217.708816,27.593082,...,9.135700e+07,6.391332e+07,1.139640e+00,2.757405e+01,4.783491e+15,4.783495e+15,4.733209e+09,1.466163e+11,5.839614e+09,4.507008e+01
min,867.000000,1.212679e+08,1.100000e+08,0.000000,900.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,2.318056e+06,-1.905360e+07,0.000000e+00,0.000000e+00,1.655472e+18,1.655472e+18,1.655472e+12,7.932016e+08,1.657470e+12,0.000000e+00
25%,22065.000000,7.656822e+08,7.900000e+08,4.000000,7767.000000,35.000000,0.000000,91.000000,0.000000,18.000000,...,1.050190e+08,7.586754e+07,0.000000e+00,1.600000e+01,1.655472e+18,1.655472e+18,1.655511e+12,5.057206e+10,1.657696e+12,0.000000e+00
50%,55211.000000,2.088317e+09,2.220000e+09,38.000000,16927.000000,78.000000,1.000000,457.000000,9.000000,36.000000,...,1.529891e+08,1.270113e+08,0.000000e+00,3.400000e+01,1.655513e+18,1.655513e+18,1.655556e+12,1.174039e+11,1.657949e+12,2.000000e+00
75%,126467.500000,4.457292e+09,5.100000e+09,260.000000,35092.000000,213.000000,8.000000,1893.000000,42.000000,61.000000,...,2.002951e+08,1.822930e+08,2.000000e+00,5.800000e+01,1.657803e+18,1.657803e+18,1.657817e+12,2.488826e+11,1.670206e+12,4.000000e+00
max,606584.000000,8.541941e+11,5.120000e+10,105185.000000,324594.000000,4143.000000,112.000000,61746.000000,3560.000000,104.000000,...,4.261504e+08,3.330271e+08,3.000000e+00,1.000000e+02,1.670665e+18,1.670666e+18,1.670599e+12,6.030459e+11,1.670572e+12,8.540000e+02


In [9]:
df_runtime = run_time(df_raw)
df_runtime[DF.run_time_per_iteration] = (
    df_runtime[DF.run_time] / df_runtime[DF.iteration_count]
)

df_runtime_average = (
    df_runtime.groupby(BENCHMARK_ENV_COL)[DF.run_time_per_iteration]
    .mean()
    .reset_index()
)

df_pivot = df_runtime.pivot_table(
    index=[RF.suite, RF.benchmark, DF.env],
    columns=[RF.type],
    values=[DF.run_time_per_iteration],
).reset_index()
df_pivot[DF.run_time_speedup] = (
    df_pivot[DF.run_time_per_iteration]["seqn"]
    / df_pivot[DF.run_time_per_iteration]["duet"]
) - 1

for suite in unique_suites:
    # Benchmark run duration
    fig = px.box(
        df_runtime[df_runtime[RF.suite] == suite],
        x=RF.benchmark,
        y=DF.run_time,
        color=DF.env,
        facet_col=RF.type,
        title=f"Benchmark Run Time - {suite}",
    )
    fig.update_xaxes(showticklabels=True)
    fig.show()

    # Benchmark run duration
    fig = px.box(
        df_runtime[df_runtime[RF.suite] == suite],
        x=RF.benchmark,
        y=DF.run_time_per_iteration,
        color=DF.env,
        facet_col=RF.type,
        title=f"Benchmark Run Time per iteration - {suite}",
    )
    fig.update_xaxes(showticklabels=True)
    fig.show()

    fig = px.bar(
        df_pivot[df_pivot[RF.suite] == suite],
        x=RF.benchmark,
        y=DF.run_time_speedup,
        color=DF.env,
        barmode="group",
        title=f"Run time speedup async duet vs. sequential - {suite}",
    )
    fig.show()

In [10]:
px.box(
    df_pivot, x=DF.env, y=DF.run_time_speedup, color=RF.suite, hover_data=[RF.benchmark]
)

In [11]:
df = run_time(df_raw)

# Suite run duration
df = (
    df.groupby(by=[RF.suite, RF.type, DF.env])
    .agg(suite_duration=(DF.run_time, sum), iteration_count=("iteration_count", sum))
    .reset_index()
)
fig = px.bar(
    df.sort_values(by=RF.type),
    x="suite_duration",
    y=RF.suite,
    color=RF.type,
    facet_row=DF.env,
    barmode="group",
    orientation="h",
    title="Suite Run Duration",
)
fig.update_layout(height=1000)
fig.show()

# Suite run duration per iteration
df[DF.run_time_per_iteration] = df["suite_duration"] / df["iteration_count"]
fig = px.bar(
    df.sort_values(by=RF.type),
    x=DF.run_time_per_iteration,
    y=RF.suite,
    color=RF.type,
    facet_row=DF.env,
    barmode="group",
    orientation="h",
    title="Suite Run Duration per iteration",
)
# fig.update_layout(height=1000)
fig.show()

# Suite speedup per iteration
df = df.pivot_table(
    index=[RF.suite, DF.env], columns=[RF.type], values=[DF.run_time_per_iteration]
).reset_index()
df[DF.run_time_speedup] = (
    df[DF.run_time_per_iteration]["seqn"] / df[DF.run_time_per_iteration]["duet"]
) - 1

df["text"] = df[DF.run_time_speedup].apply(lambda x: f"{(x * 100):.0f} %")
fig = px.bar(
    df,
    y=RF.suite,
    x=DF.run_time_speedup,
    color=DF.env,
    barmode="group",
    text="text",
)
fig.update_layout(
    xaxis_title="Average runtime speedup per iteration",
    yaxis_title="Suite",
    legend_title="Environment",
)
# fig.write_image(f"figures/runtime_speedup.pdf")
fig.show()